# Scratchwork: Cayley-Dickson Algebras

The purpose of this notebook is to develop the functions required to perform the Cayley-Dickson construction within the ``finite_algebras`` module.

In [1]:
import finite_algebras as alg
import numpy as np

## The Cayley-Dickson Construction

The [Cayley-Dickson construction](https://en.wikipedia.org/wiki/Cayley%E2%80%93Dickson_construction) begins with a <i>base</i> algebra, $A$, usually a ring or field, from which another algebra, $\mathscr{C}(A)$, is derived, with compound elements, identical to those found in the direct product of $A$ with itself, and, like direct products, addition is defined element-wise, but multiplication is defined similar to complex number multiplication, as described in definition 1, below.

> **Definition 1: Cayley-Dickson Algebra (CDA)**
> 
> Let $A = \langle S, +, \cdot \rangle$ be an algebra with <i>addition</i> & <i>multiplication</i> operations, $+$ & $\cdot$, resp., over a set of elements, $S$,
> 
> Then, let $T = S \times S$ (i.e., the cross-product of $S$ with itself)
> 
> and define the algebra, $\mathscr{C}(A) \equiv \langle T, \oplus, \odot \rangle$,
> 
> where $\forall (a,b),(c,d) \in T$,
> 
> $(a,b) \oplus (c,d) \equiv (a + c, b + d)$
> 
> $(a,b) \odot (c,d) \equiv (a \cdot c - b \cdot d, a \cdot d + b \cdot c)$

So, for example, if we apply the Cayley-Dickson construction to the field of real numbers, $\mathbb{R}$, we obtain the algebra, $\mathscr{C}(\mathbb{R}) \equiv \mathbb{C}$, of **complex numbers**.
 
The Cayley-Dickson Algebra (CDA) constructor, $\mathscr{C}$, can be applied to another CDA, and so on, and so on, according to definition 2, below.

> **Definition 2: CDA Constructor**
> 
> Let $k \in Z^+$ and $\mathscr{C}^0(A) \equiv A$,
> 
> then $\mathscr{C}^k(A) \equiv \mathscr{C}(\mathscr{C}^{k-1}(A))$,

So, for example, $\mathscr{C}^2(\mathbb{R}) \equiv \mathscr{C}(\mathbb{C}) \equiv \mathbb{H}$ is the algebra of **quaternions**, and $\mathscr{C}^3(\mathbb{R}) \equiv \mathbb{O}$ is the algebra of **octonions**.

In many (all?) papers on the subject, a generalization of complex multiplication is used, instead of the multiplication described above.

The generalization begins with a recursive definition of <i>conjugation</i>, described in definition 3, below.

> **Definition 3: Conjugation**
> 
> The **conjugate** of an algebraic element, $a$, is denoted by $\overline{a}$, and defined as follows:
> 
> For $a \in A$, the base algebra, let $\overline{a} \equiv a$.
> 
> Then, let $k \in Z^+$; $p,q \in \mathscr{C}^{k-1}(A)$; and $(p,q) \in \mathscr{C}^k(A)$,
> 
> and define $\overline{(p, q)} \equiv (\overline{p}, -q)$

In **Shafer (1966)**, the generalization of CDA multiplication is defined as shown in definition 4, below.

> **Definition 4**
> 
> Let $k \in Z^+$; $a,b,c,d,\mu \in \mathscr{C}^{k-1}(A)$; and $\mu \neq 0$,
> 
> then define multiplication for $(a,b), (c,d) \in \mathscr{C}^k(A)$
>
> as $(a, b) \odot (c, d) = (a \cdot c + \mu \cdot d \cdot \overline{b}, \overline{a} \cdot d + c \cdot b)$,

## Bibliography

* Schafer, R. D., "An Introduction to Nonassociative Algebras", Academic Press, 1966. ([or see this 1961 version](https://www.gutenberg.org/ebooks/25156))
* [Cayley Dickson algebra implementation in python](https://github.com/thoppe/Cayley-Dickson/blob/master/src/cayley_dickson.py) - github repo
* [Python code for octonion and sedenion multiplication](https://www.johndcook.com/blog/2018/07/09/octonioin-multiplication/) - John D Cook blog
* [The Octonions](https://web.archive.org/web/20180216125124/http://math.ucr.edu:80/home/baez/octonions/) - The Wayback Machine
* [Algebra over a field](https://en.wikipedia.org/wiki/Algebra_over_a_field) - Wikipedia
* [Cayley-Dickson construction](https://en.wikipedia.org/wiki/Cayley%E2%80%93Dickson_construction) - Wikipedia
* [Cayley-Dickson algebra](https://encyclopediaofmath.org/wiki/Cayley-Dickson_algebra) - Encyclopedia of Math
* [What comes after the ducentiquinquagintasexions?](https://english.stackexchange.com/questions/234607/what-comes-after-the-ducentiquinquagintasexions) - StackExchange
* ["Equivalence in a Class of Division Algebras of Order 16"](https://core.ac.uk/reader/82141950) by R. D. Schafer

## Algebras

### Algebras by "Squaring"

In [2]:
F3 = alg.generate_algebra_mod_n(3, elem_name='')
F3sqr = F3.sqr()

F4 = alg.generate_algebra_mod_n(4, elem_name='')
F4sqr = F4.sqr()

F5 = alg.generate_algebra_mod_n(5, elem_name='')
F5sqr = F5.sqr()

F7 = alg.generate_algebra_mod_n(7, elem_name='')
F7sqr = F7.sqr()

In [3]:
F3.about()


** Field **
Name: F3
Instance ID: 4603876816
Description: Autogenerated Field of integers mod 3
Order: 3
Identity: '0'
Commutative? Yes
Cyclic?: Yes
  Generators: ['1', '2']
Elements:
   Index   Name   Inverse  Order
      0     '0'     '0'       1
      1     '1'     '2'       3
      2     '2'     '1'       3
Cayley Table (showing indices):
[[0, 1, 2], [1, 2, 0], [2, 0, 1]]
Mult. Identity: '1'
Mult. Commutative? Yes
Zero Divisors: None
Multiplicative Cayley Table (showing indices):
[[0, 0, 0], [0, 1, 2], [0, 2, 1]]


### One Application of Cayley-Dickson

Here, one application of Cayley-Dickson construction process is applied to the F3 & F5 fields.

Note:
3 - Gaussian prime
4 - Not a prime
5 - Prime, but not a Gaussian prime

**Conclusion**: F3sqr, F3cda, F3cda66, & F3cda53 are all identical. Similar results for F4 & F5.

In [4]:
F3cda = F3.make_cayley_dickson_algebra()
print(f"{F3cda.description}")
print(f"F3sqr == F3cda ?: {F3sqr == F3cda}\n")

F3cda66 = F3.make_cayley_dickson_algebra(version=1)
print(f"{F3cda66.description}")
print(f"F3sqr == F3cda66 ?: {F3sqr == F3cda66}\n")

F3cda53 = F3.make_cayley_dickson_algebra(version=2)
print(f"{F3cda53.description}")
print(f"F3sqr == F3cda53 ?: {F3sqr == F3cda53}\n")

Cayley-Dickson algebra based on F3, where mu = None, basic version.
F3sqr == F3cda ?: True

Cayley-Dickson algebra based on F3, where mu = 2, Schafer 1966 version.
F3sqr == F3cda66 ?: True

Cayley-Dickson algebra based on F3, where mu = 2, Schafer 1953 version.
F3sqr == F3cda53 ?: True



In [5]:
F4cda = F4.make_cayley_dickson_algebra()
print(f"{F4cda.description}")
print(f"F4sqr == F4cda ?: {F4sqr == F4cda}\n")

F4cda66 = F4.make_cayley_dickson_algebra(version=1)
print(f"{F4cda66.description}")
print(f"F4sqr == F4cda66 ?: {F4sqr == F4cda66}\n")

F4cda53 = F4.make_cayley_dickson_algebra(version=2)
print(f"{F4cda53.description}")
print(f"F4sqr == F4cda53 ?: {F4sqr == F4cda53}\n")

Cayley-Dickson algebra based on R4, where mu = None, basic version.
F4sqr == F4cda ?: True

Cayley-Dickson algebra based on R4, where mu = 3, Schafer 1966 version.
F4sqr == F4cda66 ?: True

Cayley-Dickson algebra based on R4, where mu = 3, Schafer 1953 version.
F4sqr == F4cda53 ?: True



In [6]:
F5cda = F5.make_cayley_dickson_algebra()
print(f"{F5cda.description}")
print(f"F5sqr == F5cda ?: {F5sqr == F5cda}\n")

F5cda66 = F5.make_cayley_dickson_algebra(version=1)
print(f"{F5cda66.description}")
print(f"F5sqr == F5cda66 ?: {F5sqr == F5cda66}\n")

F5cda53 = F5.make_cayley_dickson_algebra(version=2)
print(f"{F5cda53.description}")
print(f"F5sqr == F5cda53 ?: {F5sqr == F3cda53}\n")

Cayley-Dickson algebra based on F5, where mu = None, basic version.
F5sqr == F5cda ?: True

Cayley-Dickson algebra based on F5, where mu = 4, Schafer 1966 version.
F5sqr == F5cda66 ?: True

Cayley-Dickson algebra based on F5, where mu = 4, Schafer 1953 version.
F5sqr == F5cda53 ?: False



### Two Applications of Cayley-Dickson

In [7]:
%%time

F3quad = F3sqr.sqr()
#print(F3quad)

CPU times: user 726 ms, sys: 1.15 ms, total: 727 ms
Wall time: 727 ms


In [8]:
%%time

F3cda2 = F3cda.make_cayley_dickson_algebra()

F3cda2_66 = F3cda.make_cayley_dickson_algebra(version=1)

F3cda2_53 = F3cda.make_cayley_dickson_algebra(version=2)

CPU times: user 2.24 s, sys: 1.9 ms, total: 2.24 s
Wall time: 2.24 s


In [9]:
print(F3cda2)
print(f"F3cda2 == F3quad ?: {F3cda2 == F3quad} (This should be True)")
print(f"F3cda2 == F3cda2_66 ?: {F3cda2 == F3cda2_66}")
print(f"F3cda2 == F3cda2_53 ?: {F3cda2 == F3cda2_53}")
print(f"F3cda2_66 == F3cda2_53 ?: {F3cda2_66 == F3cda2_53}")

<Ring:F3_Sqr_Sqr, ID:4684676752>
F3cda2 == F3quad ?: True (This should be True)
F3cda2 == F3cda2_66 ?: False
F3cda2 == F3cda2_53 ?: False
F3cda2_66 == F3cda2_53 ?: False


In [10]:
print(len(F3quad.zero_divisors()))
print(len(F3cda2.zero_divisors()))
print(len(F3cda2_66.zero_divisors()))
print(len(F3cda2_53.zero_divisors()))

16
16
32
32


In [11]:
alg1 = F3cda2_66
alg2 = F3cda2_53

total = 0
count = 0
for a in alg1:
    for b in alg1:
        total += 1
        if not alg1.mult(a, b) == alg2.mult(a, b):
            count += 1
print(f"{count} unequal out of {total}")

5616 unequal out of 6561


In [12]:
alg1 = F3cda2_66
alg2 = F3cda2

total = 0
count = 0
for a in alg1:
    for b in alg1:
        total += 1
        if not alg1.mult(a, b) == alg2.mult(a, b):
            count += 1
print(f"{count} unequal out of {total}")

5184 unequal out of 6561


In [13]:
alg1 = F3cda2
alg2 = F3cda2_53

total = 0
count = 0
for a in alg1:
    for b in alg1:
        total += 1
        if not alg1.mult(a, b) == alg2.mult(a, b):
            count += 1
print(f"{count} unequal out of {total}")

5184 unequal out of 6561


## Norm Squared

This tests that the following relationship holds true: $\forall x, y \in F, N(x) \cdot N(y) = N(xy)$

In [14]:
A = F3cda2
ok = 0
not_ok = 0
for x in A:
    for y in A:
        if not (A.mult(A.norm_sqr(x), A.norm_sqr(y)) == A.norm_sqr(A.mult(x, y))):
            not_ok += 1
            All_OK = False
            print(x, y)
        else:
            ok += 1

print(f"{A.name}: {ok} OK, {not_ok} not OK")

F3_Sqr_Sqr: 6561 OK, 0 not OK


In [15]:
algebras = [F3, F3sqr, F3quad, F3cda, F3cda66, F3cda53, F3cda2, F3cda2_66, F3cda2_53]

for A in algebras:
    ok = 0
    not_ok = 0
    for x in A:
        for y in A:
            if not (A.mult(A.norm_sqr(x), A.norm_sqr(y)) == A.norm_sqr(A.mult(x, y))):
                not_ok += 1
                All_OK = False
                # print(x, y)
            else:
                ok += 1
    print(f"{A.name}: {ok} OK, {not_ok} not OK")

F3: 9 OK, 0 not OK
F3_SQR: 81 OK, 0 not OK
F3_SQR_SQR: 6561 OK, 0 not OK
F3_Sqr: 81 OK, 0 not OK
F3_CDA66: 81 OK, 0 not OK
F3_CDA53: 81 OK, 0 not OK
F3_Sqr_Sqr: 6561 OK, 0 not OK
F3_Sqr_CDA66: 1265 OK, 5296 not OK
F3_Sqr_CDA53: 1265 OK, 5296 not OK


## Check Conjugate